In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from cpd_utils import *

import time
import bisect

import pandas as pd

# DCDP

In [2]:
def generate_data_mean(n, T, theta):
    p = len(theta[0])
    y_train = np.stack([np.random.multivariate_normal(theta[i], np.eye(p), n[i]) for i in range(T)])
    y_train_joint = y_train.reshape((-1, p))
    nt = len(y_train_joint)
    
    return nt, y_train_joint

In [29]:
T = 2
Delta = 50
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 1
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_mean(n, T, theta)
nt, Y_test = generate_data_mean(n, T, theta)

In [30]:
theta.shape
Y_train.shape

(100, 1)

In [31]:
grid_n = 200
gamma_list = [50, 100, 500]
lam_list = [0, 0.5]

B = 1
run_time_dc = np.zeros(B)

loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_mean(grid_n, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 0.0, avg time: 0.8014414310455322
best parameter: (0, 50)


In [32]:
print(cp_best)
print(cp_best_cand)

[50]
[50]


In [33]:
cp = np.concatenate([[0], cp_best_cand[:], [len(Y_train)]])
beta_path = dcdp.fit_with_cp(Y_train, cp)

In [34]:
print(beta_path)
print(theta)

[[5.0274945 ]
 [0.06648559]]
[[5.]
 [0.]]


In [35]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 20
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_mean(n, T, theta)
nt, Y_test = generate_data_mean(n, T, theta)

In [36]:
theta

array([[5., 5., 5., 5., 5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 5., 5., 5., 5., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 5., 5., 5., 5., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5.,
        5., 5., 5., 5.]])

In [37]:
theta.shape
Y_train.shape

(2000, 20)

In [44]:
grid_n = 100
gamma_list = [500, 1000]
lam_list = [0.5]

B = 1
run_time_dc = np.zeros(B)

loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_mean(grid_n, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 0.0, avg time: 1.1623988151550293
best parameter: (0.5, 500)


In [45]:
print(cp_best)
print(cp_best_cand)

[500, 1000, 1500]
[ 500 1000 1493]


In [40]:
cp = np.concatenate([[0], cp_best_cand[:], [len(Y_train)]])
beta_path = dcdp.fit_with_cp(Y_train, cp)

In [41]:
beta_path

array([[ 4.90638627e+00,  4.94207691e+00,  4.94896493e+00,
         4.91494963e+00,  4.96024907e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00],
       [ 3.01904316e+00,  2.74034664e+00,  2.70405488e+00,
         2.91666343e+00,  2.97767244e+00,  1.98951804e+00,
         1.87475948e+00,  1.85296906e+00,  2.14509635e+00,
         1.70960058e+00, -0.00000000e+00, -0.00000000e+00,
         3.85473677e-02,  0.00000000e+00,  0.00000000e+00,
        -6.12443920e-03, -9.85621418e-02, -0.00000000e+00,
         0.00000000e+00,  3.59952783e-03],
       [-0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00,  4.85760508e+00,
         4.93640410e+00,  4.96116553e+00,  4.81042650e+00,
         4.83858685e+00, -0.0

In [46]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_mean(n, T, theta)
nt, Y_test = generate_data_mean(n, T, theta)

In [49]:
grid_n = 100
gamma_list = [2000]
lam_list = [0.1, 0.5]

B = 1
run_time_dc = np.zeros(B)

loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_mean(grid_n, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 0.0, avg time: 3.5858418941497803
best parameter: (0.1, 2000)


In [50]:
print(cp_best)
print(cp_best_cand)

[500, 1000, 1500]
[ 499  983 1483]


In [15]:
cp = np.concatenate([[0], cp_best_cand[:], [len(Y_train)]])
beta_path = dcdp.fit_with_cp(Y_train, cp, 0.1)

In [14]:
# beta_path

Weaker signal

In [51]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 1

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_mean(n, T, theta)
nt, Y_test = generate_data_mean(n, T, theta)

In [52]:
grid_n = 100
gamma_list = [200, 800, 2000]
lam_list = [0.1]

B = 1
run_time_dc = np.zeros(B)

loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_mean(grid_n, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 0.0, avg time: 5.608494520187378
best parameter: (0.1, 200)


In [53]:
print(cp_best)
print(cp_best_cand)

[500, 1000, 1500]
[ 485 1010 1531]


In [68]:
T = 4
Delta = 50
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 1

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_mean(n, T, theta)
nt, Y_test = generate_data_mean(n, T, theta)

In [69]:
grid_n = 100
gamma_list = [200, 800, 2000]
lam_list = [0.1]

B = 1
run_time_dc = np.zeros(B)

loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_mean(grid_n, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 0.0, avg time: 0.6674776077270508
best parameter: (0.1, 200)


In [70]:
print(cp_best)
print(cp_best_cand)

[50, 100, 150]
[ 51 100 150]


In [82]:
T = 4
Delta = 200
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 0.5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_mean(n, T, theta)
nt, Y_test = generate_data_mean(n, T, theta)

In [87]:
grid_n = 100
gamma_list = [200, 800, 2000]
lam_list = [0.1]

B = 1
run_time_dc = np.zeros(B)

loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_mean(grid_n, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 4.0, avg time: 1.474365472793579
best parameter: (0.1, 200)


In [88]:
print(cp_best)
print(cp_best_cand)

[202, 396, 599]
[203 376 582]


## Check run_time and loc_error w.r.t. Q_grid

In [ ]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

theta = np.array([0, 5, 0, 5])

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_univ_mean(n, T, theta)
nt, Y_test = generate_data_univ_mean(n, T, theta)

Q_grid_list = [25,50,75,100,125,150,175,200]
Q = len(Q_grid_list)

gamma_list = [40, 80, 120]
lam_list = [None]

B = 100

run_time_d = np.zeros((Q,B))
run_time_dc = np.zeros((Q,B))

loc_error_d = np.zeros((Q,B))
loc_error_dc = np.zeros((Q,B))

print('---------- only divide -----------')
for q, grid_n in enumerate(Q_grid_list):
    for b in range(B):
        start_time = time.time()
        cp_best, param_best = dp_cv_random_mean((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_d[q, b] = cp_distance(cp_best, cp_truth)
        run_time_d[q, b] = time.time() - start_time
        
print('---------- divide and conquer -----------')
for q, grid_n in enumerate(Q_grid_list):
    for b in range(B):
        start_time = time.time()
        cp_best, param_best, cp_best_cand = dcdp_random((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_dc[q, b] = cp_distance(cp_best, cp_truth)
        run_time_dc[q, b] = time.time() - start_time

In [ ]:
import pickle
with open('Q_time_error_4.pickle', 'wb') as f:
    pickle.dump([theta, Y_train, Y_test, Delta, Q_grid_list, gamma_list, run_time_d, run_time_dc, loc_error_d, loc_error_dc], f)

In [ ]:
# objects = []
# with (open("Q_time_error_1.pickle", "rb")) as openfile:
#     while True:
#         try:
#             objects.append(pickle.load(openfile))
#         except EOFError:
#             break

In [ ]:
def curve_with_bar(x, y_list, percent, legend, xlabel, ylabel, save = False, name = None):
    plt.figure(figsize = (10,7))
    for y in y_list:
        plt.fill_between(x, np.quantile(y, percent, axis = 1), np.quantile(y, 1 - percent, axis = 1), alpha = 0.2)
        plt.plot(x, y.mean(axis = 1))

    plt.legend(legend)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if save:
        plt.savefig(name)

In [ ]:
curve_with_bar(Q_grid_list, [loc_error_d, loc_error_dc], 0.1, ['divide','divide and conquer'], 'grid num', 'loc error')

In [ ]:
plt.hist(loc_error_d[1,:], alpha = 0.5)
plt.hist(loc_error_dc[1,:], alpha = 0.5)
plt.legend(['divide','divide and conquer'])

In [ ]:
plt.hist(loc_error_d[-1,:], alpha = 0.5)
plt.hist(loc_error_dc[-1,:], alpha = 0.5)
plt.legend(['divide','divide and conquer'])

In [ ]:
curve_with_bar(Q_grid_list, [run_time_d, run_time_dc], 0.1, ['divide','divide and conquer'], 'grid num', 'run time')

In [ ]:
import winsound
duration = 10000  # milliseconds
freq_base = 440  # Hz
index_base = 49
winsound.Beep(freq_base, duration)

In [ ]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

theta = np.array([0, 5, 0, 5])

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

Q_grid_list = [25,50,75,100,125,150,175,200]
Q = len(Q_grid_list)

gamma_list = [10, 20, 40, 80]
gamma_list2 = [80]
lam_list = [None]

B = 100

run_time_d = np.zeros((Q,B))
loc_error_d = np.zeros((Q,B))

run_time_d2 = np.zeros((Q,B))
loc_error_d2 = np.zeros((Q,B))

run_time_dc = np.zeros((Q,B))
loc_error_dc = np.zeros((Q,B))

run_time_dc2 = np.zeros((Q,B))
loc_error_dc2 = np.zeros((Q,B))

for q, grid_n in enumerate(Q_grid_list):
    for b in range(B):
        nt, Y_train = generate_data_univ_mean(n, T, theta)
        nt, Y_test = generate_data_univ_mean(n, T, theta)
        
        start_time = time.time()
        cp_best, param_best = dp_cv_random_mean((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_d[q, b] = cp_distance(cp_best, cp_truth)
        run_time_d[q, b] = time.time() - start_time
        
        # check local refinement
        start_time = time.time()
        cp_best, param_best, cp_best_cand = dcdp_random((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_dc[q, b] = cp_distance(cp_best, cp_truth)
        run_time_dc[q, b] = time.time() - start_time
        
        #### only use large value of gamma
        start_time = time.time()
        cp_best, param_best = dp_cv_random_mean((Y_train), (Y_test), grid_n, lam_list, gamma_list2, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_d2[q, b] = cp_distance(cp_best, cp_truth)
        run_time_d2[q, b] = time.time() - start_time
        
        # local refinement, large value of gamma
        start_time = time.time()
        cp_best, param_best, cp_best_cand = dcdp_random((Y_train), (Y_test), grid_n, lam_list, gamma_list2, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_dc2[q, b] = cp_distance(cp_best, cp_truth)
        run_time_dc2[q, b] = time.time() - start_time

In [ ]:
import pickle
with open('Q_time_error_5.pickle', 'wb') as f:
    pickle.dump([theta, Y_train, Y_test, Delta, Q_grid_list, gamma_list, run_time_d, run_time_dc, loc_error_d, loc_error_dc], f)

In [ ]:
# objects = []
# with (open("Q_time_error_1.pickle", "rb")) as openfile:
#     while True:
#         try:
#             objects.append(pickle.load(openfile))
#         except EOFError:
#             break

In [ ]:
def curve_with_bar(x, y_list, percent, legend, xlabel, ylabel, save = False, name = None):
    plt.figure(figsize = (10,7))
    for y in y_list:
        plt.fill_between(x, np.quantile(y, percent, axis = 1), np.quantile(y, 1 - percent, axis = 1), alpha = 0.2)
        plt.plot(x, y.mean(axis = 1))
    
    fsize = 20
    plt.legend(legend)
    plt.xlabel(xlabel, fontsize = fsize)
    plt.ylabel(ylabel, fontsize = fsize)
    if save:
        plt.savefig(name)

In [ ]:
curve_with_bar(Q_grid_list, [loc_error_d, loc_error_dc], 0.1, ['divide','divide and conquer'], 'grid num', 'loc error')

In [ ]:
plt.hist(loc_error_d[1,:], alpha = 0.5)
plt.hist(loc_error_dc[1,:], alpha = 0.5)
plt.legend(['divide','divide and conquer'])

In [ ]:
plt.hist(loc_error_d[-1,:], alpha = 0.5)
plt.hist(loc_error_dc[-1,:], alpha = 0.5)
plt.legend(['divide','divide and conquer'])

In [ ]:
curve_with_bar(Q_grid_list, [run_time_d, run_time_dc], 0.1, ['divide','divide and conquer'], 'grid num', 'run time')

In [ ]:
curve_with_bar(Q_grid_list, [run_time_d], 0.1, [], 'grid size', 'run time')
plt.savefig('run_time_Delta500_B100.pdf',bbox_inches='tight')

In [ ]:
import winsound
duration = 10000  # milliseconds
freq_base = 440  # Hz
index_base = 49
winsound.Beep(freq_base, duration)

In [ ]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

theta = np.array([0, 5, 0, 5])

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

grid_n = 100


gamma_list_ = [5, 10, 20, 40, 80, 100, 150, 200, 300]
lam_list = [None]

Q = len(gamma_list_)
B = 40

run_time_d = np.zeros((Q,B))
run_time_dc = np.zeros((Q,B))

loc_error_d = np.zeros((Q,B))
loc_error_dc = np.zeros((Q,B))

for q, gamma in enumerate(gamma_list_):
    gamma_list = [gamma]
    for b in range(B):
        nt, Y_train = generate_data_univ_mean(n, T, theta)
        nt, Y_test = generate_data_univ_mean(n, T, theta)
        
        start_time = time.time()
        cp_best, param_best = dp_cv_random_mean((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_d[q, b] = cp_distance(cp_best, cp_truth)
        run_time_d[q, b] = time.time() - start_time

        start_time = time.time()
        cp_best, param_best, cp_best_cand = dcdp_random((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                              loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
        loc_error_dc[q, b] = cp_distance(cp_best, cp_truth)
        run_time_dc[q, b] = time.time() - start_time

In [ ]:
import pickle
with open('gamma_time_error_3.pickle', 'wb') as f:
    pickle.dump([theta, Y_train, Y_test, Delta, Q_grid_list, gamma_list, run_time_d, run_time_dc, loc_error_d, loc_error_dc], f)

In [ ]:
# objects = []
# with (open("Q_time_error_1.pickle", "rb")) as openfile:
#     while True:
#         try:
#             objects.append(pickle.load(openfile))
#         except EOFError:
#             break

In [ ]:
curve_with_bar(gamma_list_, [loc_error_d, loc_error_dc], 0.1, ['divide','divide and conquer'], 'gamma', 'loc error')

In [ ]:
plt.hist(loc_error_d[1,:], alpha = 0.5)
plt.hist(loc_error_dc[1,:], alpha = 0.5)
plt.legend(['divide','divide and conquer'])

In [ ]:
plt.hist(loc_error_d[-1,:], alpha = 0.5)
plt.hist(loc_error_dc[-1,:], alpha = 0.5)
plt.legend(['divide','divide and conquer'])

In [ ]:
curve_with_bar(gamma_list_, [run_time_d, run_time_dc], 0.1, ['divide','divide and conquer'], 'gamma', 'run time')

In [ ]:
import winsound
duration = 10000  # milliseconds
freq_base = 440  # Hz
index_base = 49
winsound.Beep(freq_base, duration)

In [ ]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

Y_train = pd.read_csv('y_train.csv', header = None).values.squeeze()
Y_test = pd.read_csv('y_test.csv', header = None).values.squeeze()

nt = len(Y_train)

gamma_list = [10, 20, 40, 100]
# gamma_list = 20 * nt * np.arange(0.0001,0.01,0.0005)
lam_list = [None]


grid_n = 50
print('---------- only divide -----------')

start_time = time.time()

cp_best, param_best = dp_cv_random_mean((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                      loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
run_time = time.time() - start_time
loc_error = cp_distance(cp_best, cp_truth)

print(cp_best)
print(param_best)
print("loc error: {0}, time: {1}".format(loc_error, run_time))


print('---------- divide and conquer -----------')
start_time = time.time()

cp_best, param_best, cp_best_cand = dcdp_random((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                      loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
run_time = time.time() - start_time
loc_error = cp_distance(cp_best, cp_truth)

print(cp_best)
print(param_best)
print("loc error: {0}, time: {1}".format(loc_error, run_time))

In [ ]:
# Y_train = np.array([0,0,0,1,1,1,2,2,2])
# Y_test = np.array([0,0,0,1,1,1,2,2,2])
Y_train = np.repeat(np.array([0,5,10,5]),500)
Y_test = np.repeat(np.array([0,5,10,5]),500)
cp_loc = np.array([3,6])
g = goodness_of_fit_univ_mean(Y_train, Y_test, cp_loc, None, 10, estim_univ_mean, loss_univ_mean)
print(g)

lam_list = [None]
# gamma_list = [0.3,0.5,1,2,4]
# gamma_list = [1,2,3,4,5]
gamma_list = [4,50,100]
# gamma_list = [1,2,3,4,5]

nt = len(Y_train)
# grid = np.sort(np.random.choice(nt, 5 * 50, replace = False))
# print(grid)
# grid = np.sort(np.arange(0,nt,30))
grid_n = 50
cp_best, param_best = dp_cv_random_mean((Y_train), (Y_test), grid_n, lam_list, gamma_list, 
                                      loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)

print(cp_best)
print(param_best)

In [ ]:
4.39 * 2 + 7.69 + 8.39

In [ ]:
np.mean(np.ones(10))

In [ ]:
T = 3
n = 50
m = np.array([5000, 5000, 5000])
cp_truth = np.cumsum(m)[:T-1]

beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_t = get_beta_with_gap(n, delta)
beta_t *= kappa / (np.max(beta_t) - np.min(beta_t))
beta[0] = beta_t[:]
beta[1] = beta_t[np.random.permutation(n)]
beta[2] = beta[1][np.random.permutation(n)]

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

np.random.seed(0)

nt, X_train, Y_train = generate_data_bt(n, m, T, beta)
nt, X_test, Y_test = generate_data_bt(n, m, T, beta)

In [ ]:
grid = np.arange(0, nt, step = 200)[1:]
gamma_list = [10 * np.log(nt * n), 20 * np.log(nt * n), 40 * np.log(nt * n)]
lam_list = [0.1, 0.5, 1]

cp_best, param_best = dp_cv_grid_covariate((X_train, Y_train), (X_test, Y_test), grid, lam_list, gamma_list, bt_loss, bt_newton_solver)

In [ ]:
cp_best

In [ ]:
gamma_list

In [ ]:
param_best

# DCDP with fixed grid

In [ ]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

np.random.seed(0)

sig = 5
theta = sig * np.random.normal(0, 1, T)

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt, Y_train = generate_data_univ_mean(n, T, theta)
nt, Y_test = generate_data_univ_mean(n, T, theta)

# grid = np.arange(0, nt, step = Delta // 10)[1:]
grid = np.sort(np.random.choice(nt, 50 * nt // Delta, replace = False))
gamma_list = [10, 20]
lam_list = [None]

start_time = time.time()

cp_best, param_best = dp_cv_grid_mean((Y_train), (Y_test), grid, lam_list, gamma_list, 
                                      loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
run_time = time.time() - start_time
loc_error = cp_distance(cp_best, cp_truth)


print("loc error: {0}, time: {1}".format(loc_error, run_time))

In [ ]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

np.random.seed(0)

sig = 5
theta = sig * np.random.normal(0, 1, T)

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt, Y_train = generate_data_univ_mean(n, T, theta)
nt, Y_test = generate_data_univ_mean(n, T, theta)

# grid = np.arange(0, nt, step = Delta // 10)[1:]
Q_grid = np.array([5, 10, 20, 30, 40, 50])

run_time_list = np.zeros_like(Q_grid)
loc_error_list = np.zeros_like(Q_grid)

for i, q in enumerate(Q_grid):
    grid = np.sort(np.random.choice(nt, q * nt // Delta, replace = False))
    gamma_list = [10, 20]
    lam_list = [None]

    start_time = time.time()

    cp_best, param_best = dp_cv_grid_mean((Y_train), (Y_test), grid, lam_list, gamma_list, 
                                          loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
    run_time = time.time() - start_time
    loc_error = cp_distance(cp_best, cp_truth)
    
    run_time_list[i] = run_time
    loc_error_list[i] = loc_error

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (10, 4))
ax[0].plot(Q_grid, run_time_list)
ax[1].plot(Q_grid, loc_error_list)

In [ ]:
ax

In [ ]:
np.concatenate([[0],np.arange(3)])

In [ ]:
theta

In [ ]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

np.random.seed(1000)

# sig = 5
# theta = sig * np.random.normal(0, 1, T)
theta = np.array([0, -2, 10, 1])

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt, Y_train = generate_data_univ_mean(n, T, theta)
nt, Y_test = generate_data_univ_mean(n, T, theta)

# grid = np.arange(0, nt, step = Delta // 10)[1:]
# grid = np.sort(np.concatenate([np.random.choice(nt, 15 * nt // Delta, replace = False), [512, 1001, 1494]]))
grid = np.sort(np.random.choice(nt, 15 * nt // Delta, replace = False))
gamma_list = [1, 50, 200, 400]
lam_list = [None]

print('---------- only divide -----------')

start_time = time.time()

cp_best, param_best = dp_cv_grid_mean((Y_train), (Y_test), grid, lam_list, gamma_list, 
                                      loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
run_time = time.time() - start_time
loc_error = cp_distance(cp_best, cp_truth)

print(cp_best)
print(param_best)
print("loc error: {0}, time: {1}".format(loc_error, run_time))

print('---------- divide and conquer -----------')
start_time = time.time()

cp_best, param_best, cp_best_cand = dcdp((Y_train), (Y_test), grid, lam_list, gamma_list, 
                                      loss_univ_mean, estim_univ_mean, goodness_of_fit_univ_mean)
run_time = time.time() - start_time
loc_error = cp_distance(cp_best, cp_truth)

print(cp_best)
print(param_best)
print("loc error: {0}, time: {1}".format(loc_error, run_time))

In [ ]:
grid

In [ ]:
cp_best_cand

## larger T

In [ ]:
T = 3
n = 50
m = np.array([10000, 10000, 10000])
cp_truth = np.cumsum(m)[:T-1]

beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_t = get_beta_with_gap(n, delta)
beta_t *= kappa / (np.max(beta_t) - np.min(beta_t))
beta[0] = beta_t[:]
beta[1] = beta_t[np.random.permutation(n)]
beta[2] = beta[1][np.random.permutation(n)]

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

np.random.seed(0)

nt, X_train, Y_train = generate_data_bt(n, m, T, beta)
nt, X_test, Y_test = generate_data_bt(n, m, T, beta)

In [ ]:
grid = np.arange(0, nt, step = 2000)[1:]
gamma_list = [20 * np.log(nt * n), 40 * np.log(nt * n), 60 * np.log(nt * n)]
lam_list = [0.1, 0.5, 1]

cp_best, param_best = dp_cv_grid_covariate((X_train, Y_train), (X_test, Y_test), grid, lam_list, gamma_list, bt_loss, bt_newton_solver)

In [ ]:
cp_best

In [ ]:
gamma_list

In [ ]:
param_best

In [ ]:
grid = np.arange(0, nt, step = 2000)[1:]
gamma = 40 * np.log(nt * n)
lamb = 0.1

cp_loc, B, dp = dp_grid_covariate(X_train, Y_train, grid, lamb, gamma, bt_loss, bt_newton_solver)

In [ ]:
grid

In [ ]:
cp_loc

In [ ]:
len(grid)

In [ ]:
len(dp), len(dp[0])

In [ ]:
m = len(grid)
loss_value = np.ones((m,m)) * np.infty
for i in range(m):
    for j in range(i + 1, m):
#         v = dp[j + 1][m + 1] + dp[i + 1][j + 1] + dp[0][i + 1]
        intervals = [[0, grid[i]], [grid[i], grid[j]],[grid[j], nt]]
        v = 2 * gamma
        for interval in intervals:
            l, r = interval[0], interval[1]
            Y_sub, X_sub = y_train_joint[l:r], X_train_joint[l:r]
            estimate, _ = bt_newton_solver(X_sub, Y_sub, lam)
            v += bt_loss(estimate, X_sub, Y_sub, lam)
        loss_value[i][j] = v

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(loss_value)

x_positions = np.arange(0,len(grid)) # pixel count at label position
x_labels = grid # labels you want to see
plt.xticks(x_positions, x_labels, rotation = 80)
plt.xlabel('second split')
y_positions = np.arange(0,len(grid)) # pixel count at label position
y_labels = grid # labels you want to see
plt.yticks(y_positions, y_labels)
plt.ylim(len(grid) - 0.5, -0.5)
plt.ylabel('first split')

plt.colorbar()
plt.savefig('dp_bt_2cp.pdf')

In [ ]:
B[B < np.infty]

In [ ]:
grid

In [ ]:
plt.plot(np.arange(nt)[B < np.infty], B[B < np.infty], '-')

In [ ]:
plt.figure(figsize = (10, 6))
plt.plot(cusum_val, '-')
plt.vlines(cp_truth, color = 'red', label = "true cp", ymin = min(cusum_val), ymax = max(cusum_val))
plt.vlines(cp_loc, color = 'black', linestyles = "dashed", label = "estimates", ymin = min(cusum_val), ymax = max(cusum_val))
plt.legend(["cusum", "true cp", "estimates"])

In [ ]:
beta_hat.shape

In [ ]:
n = 50

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta = get_beta_with_gap(n, delta)
beta *= kappa / (np.max(beta) - np.min(beta))

m = 10000

X_train = get_X(n, m)
y_train = np.random.binomial(1, logistic(X_train, beta))

beta_hat, l_path = bt_newton_solver(X_train, y_train)

print(logistic_beta_error(beta, beta_hat))
print(logistic_y_error(beta, beta_hat))
print(logistic_prob_error(beta, beta_hat))

In [ ]:
plt.plot(beta, 'b-')
plt.plot(beta_hat, 'r-.')
plt.legend(['beta','beta_hat'])

In [ ]:
l_path

In [ ]:
plt.plot(l_path)

In [ ]:
plt.hist(logistic(X_train, beta))

## Check magnitude of gamma

In [ ]:
def bt_loss(beta, X_train, y_train, lam):
    return -loglike_logistic(X_train, beta, y_train) + lam / 2 * np.sum(beta**2)

def bt_newton_solver(X_train, y_train, lam = 0.1, max_step = 50, tol = 1e-12, verbose = False):
    n, p = X_train.shape
    beta_hat = np.zeros((p,1))
    step_size = 1
    l_path = []
    i = 0
    
    err = 10
    a, b = 0.01, 0.3
    max_back = 100

    obj_old = np.inf
    while i < max_step and err > tol:
        g = -grad_logistic(X_train, beta_hat, y_train) + lam * beta_hat
        H = -hessian_logistic(X_train, beta_hat, y_train) + lam * np.eye(p)
        beta_new = beta_hat + 0
        s = step_size

        v = -np.linalg.solve(H, g)
        for j in range(max_back):
            beta_new = beta_new + s * v
            obj_new = -loglike_logistic(X_train, beta_new, y_train) + lam / 2 * np.sum(beta_new**2)
            if obj_new <= obj_old + b * s * g.T @ v:
                break
            s *= a
#         err = np.sum((beta_new - beta_hat)**2)**0.5
        beta_hat = beta_new
        err = np.sum((-grad_logistic(X_train, beta_hat, y_train) + lam * beta_hat)**2)**0.5
        obj_old = obj_new
        l_path.append(obj_new)
        i += 1
        if verbose:
            print("step: ",i," is done")
    if verbose:
        print('i: ', i)
        print("err: ", err)
        print("beta_hat: ", beta_hat)
    
    return beta_hat, l_path

In [ ]:
T = 2
n = 50
m = 10000

beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_t = get_beta_with_gap(n, delta)
beta_t *= kappa / (np.max(beta_t) - np.min(beta_t))
beta[0] = beta_t[:]
beta[1] = beta_t[np.random.permutation(n)]

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

# X_train = np.random.normal(0,1,(n,p))
X_train = np.stack([get_X(n, m) for _ in range(T)])
y_train = np.stack([np.random.binomial(1, logistic(X_train[i], beta[i])) for i in range(T)])

X_train_joint = X_train.reshape((-1, n))
y_train_joint = y_train.reshape((-1, 1))

In [ ]:
nt = len(y_train_joint)
cp_truth = m
lam = 0.1

def loss_loc(loc):
    X_sub, y_sub = X_train_joint[:loc], y_train_joint[:loc]
    beta_cp1, _ = bt_newton_solver(X_sub, y_sub, lam)
    loss1 = bt_loss(beta_cp1, X_sub, y_sub, lam)

    X_sub, y_sub = X_train_joint[loc:], y_train_joint[loc:]
    beta_cp2, _ = bt_newton_solver(X_sub, y_sub, lam)
    loss2 = bt_loss(beta_cp2, X_sub, y_sub, lam)
    
    return loss1 + loss2

# loss_all[cp_truth] = loss_loc(cp_truth)
step = 500
loc_list = np.arange(step, nt, step)
loss_all = np.zeros(len(loc_list))

for i, loc in enumerate(loc_list):
    loss_all[i] = loss_loc(loc)

In [ ]:
plt.plot(loc_list, loss_all, '.')

Two change points

In [ ]:
T = 3
n = 50
m = 10000

beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_t = get_beta_with_gap(n, delta)
beta_t *= kappa / (np.max(beta_t) - np.min(beta_t))
beta[0] = beta_t[:]
beta[1] = beta_t[np.random.permutation(n)]
beta[2] = beta[1][np.random.permutation(n)]

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

# X_train = np.random.normal(0,1,(n,p))
X_train = np.stack([get_X(n, m) for _ in range(T)])
y_train = np.stack([np.random.binomial(1, logistic(X_train[i], beta[i])) for i in range(T)])

X_train_joint = X_train.reshape((-1, n))
y_train_joint = y_train.reshape((-1, 1))

In [ ]:
nt = len(y_train_joint)
borda_left, borda_right = np.zeros((nt, n)), np.zeros((nt, n))

borda_left[0] = X_train_joint[0][:]
borda_right[-1] = X_train_joint[-1][:]

rank_left, rank_right = np.zeros((nt, n)), np.zeros((nt, n))

rank_left[0] = stats.rankdata(borda_left[0], method = 'min')
rank_right[-1] = stats.rankdata(borda_right[-1], method = 'min')


buffer = 1000

diff = np.zeros(nt - 1)

for i in range(1, nt):
    borda_left[i] = borda_left[i - 1] + X_train_joint[i] * (2 * y_train_joint[i] - 1)
    borda_right[nt - 1 - i] = borda_right[nt - 1 - (i - 1)] + X_train_joint[nt - 1 - i] * (2 * y_train_joint[nt - 1 - i] - 1)
    
    rank_left[i] = stats.rankdata(borda_left[i], method = 'min')
    rank_right[nt - 1 - i] = stats.rankdata(borda_right[nt - 1 - i], method = 'min')

for i in range(nt - 1):
    diff[i] = np.sum(np.abs(rank_left[i] - rank_right[i + 1])) / n

In [ ]:
diff[:30]

In [ ]:
len(diff[buffer:nt - buffer])

In [ ]:
np.arange(1,5)

In [ ]:
buffer = 0
plt.plot(np.arange(buffer, nt - 1 - buffer), diff[buffer:nt - 1 - buffer], '.')

More change points

In [ ]:
T = 4
n = 50
m = 10000

beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_t = get_beta_with_gap(n, delta)
beta_t *= kappa / (np.max(beta_t) - np.min(beta_t))
beta[0] = beta_t[:]
beta[1] = beta_t[np.random.permutation(n)]
beta[2] = beta[1][np.random.permutation(n)]

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

# X_train = np.random.normal(0,1,(n,p))
X_train = np.stack([get_X(n, m) for _ in range(T)])
y_train = np.stack([np.random.binomial(1, logistic(X_train[i], beta[i])) for i in range(T)])

X_train_joint = X_train.reshape((-1, n))
y_train_joint = y_train.reshape((-1, 1))

In [ ]:
nt = len(y_train_joint)
borda_left, borda_right = np.zeros((nt, n)), np.zeros((nt, n))

borda_left[0] = X_train_joint[0][:]
borda_right[-1] = X_train_joint[-1][:]

rank_left, rank_right = np.zeros((nt, n)), np.zeros((nt, n))

rank_left[0] = stats.rankdata(borda_left[0], method = 'min')
rank_right[-1] = stats.rankdata(borda_right[-1], method = 'min')


buffer = 1000

diff = np.zeros(nt - 1)

for i in range(1, nt):
    borda_left[i] = borda_left[i - 1] + X_train_joint[i] * (2 * y_train_joint[i] - 1)
    borda_right[nt - 1 - i] = borda_right[nt - 1 - (i - 1)] + X_train_joint[nt - 1 - i] * (2 * y_train_joint[nt - 1 - i] - 1)
    
    rank_left[i] = stats.rankdata(borda_left[i], method = 'min')
    rank_right[nt - 1 - i] = stats.rankdata(borda_right[nt - 1 - i], method = 'min')

for i in range(nt - 1):
    diff[i] = np.sum(np.abs(rank_left[i] - rank_right[i + 1]))

In [ ]:
plt.plot(np.arange(buffer, nt - buffer), diff[buffer:nt - buffer], '.')